In [1]:
from dataclasses import dataclass
from pathlib import Path
import math

import keras as tfk
import keras_cv as kcv
from keras import layers as tfkl

import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.data import Dataset as tfds

sns.set_theme()

In [3]:
SEED = 42
BATCH_SIZE = 512
METRICS = ["accuracy", "recall", "f1_score"]

tfk.utils.set_random_seed(SEED)

In [4]:
@dataclass
class Hyperparameters:
    # Model
    activation = "silu"
    # Training
    noise_std: float = 0.05
    optimiser = tfk.optimizers.Lion
    learning_rate = 3e-5
    regularization1 = tfk.regularizers.L1L2(l1=1e-5, l2=0) #1e-4
    regularization2 = tfk.regularizers.L1L2(l1=0, l2=0)
    loss = tfk.losses.CategoricalCrossentropy()
    epochs = 20
    ## Early stopping parameters
    es_patience = 10
    es_min_delta = 1e-2
    ## Learning rate schedule
    lr_patience = 5
    lr_decay_factor = 0.1
    lr_min_delta = 3e-2 
    min_lr = 1e-8

hp = Hyperparameters()

In [5]:
input_path = "/kaggle/input/augmented-blood-cells-final/augmented"
train_dataset_path = input_path + "/train"
val_dataset_path = input_path + "/val"

augmented_train_dataset = tfds.load(train_dataset_path)
augmented_val_dataset = tfds.load(val_dataset_path)

augmented_train_dataset = augmented_train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
augmented_val_dataset = augmented_val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [6]:
data_augmentation_layers = [
    tfk.layers.RandomFlip("horizontal_and_vertical"),
    tfk.layers.RandomTranslation(height_factor=0.1, width_factor=0.1, fill_mode="nearest"),  
    tfk.layers.RandomZoom(0.2, fill_mode="nearest"),
    tfk.layers.RandomRotation(0.2, fill_mode="nearest"),
    tfk.layers.RandomContrast(0.2),
    tfk.layers.RandomBrightness(factor=0.2)    
]

def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

In [8]:
def build_model(hp: Hyperparameters,
                feature_extractor: tfk.applications):
    inputs = tfkl.Input((96, 96, 3))

    x = data_augmentation(inputs)

    x = tfk.layers.Rescaling(1.0 / 255)(x)
    x = tfkl.GaussianNoise(hp.noise_std)(x)
    x = tfk.layers.Rescaling(255)(x)

    x = vgg19(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.6)(x)
    x = tfkl.Dense(512, activation=hp.activation, kernel_regularizer=hp.regularization1)(x)
    x = tfkl.Dense(256, activation=hp.activation, kernel_regularizer=hp.regularization1)(x)
    x = tfkl.Dense(128, activation=hp.activation, kernel_regularizer=hp.regularization2)(x)
    x = tfkl.Dense(64, activation=hp.activation)(x)
    x = tfkl.Dropout(0.4)(x)
    output = tfkl.Dense(8, activation="softmax")(x)

    model = tfk.Model(inputs, output)
    return model
    

In [9]:
def fit(model: tfk.Model,
        train_dataset: tf.data.Dataset,
        val_dataset: tf.data.Dataset,
        hp: Hyperparameters):
    model.compile(loss=hp.loss,
                  optimizer=hp.optimiser(learning_rate=hp.learning_rate), 
                  metrics=['accuracy'])
    
    history = model.fit(
        x=train_dataset,
        epochs=hp.epochs,
        validation_data=val_dataset,
        callbacks=[
            tfk.callbacks.EarlyStopping(monitor='val_accuracy', 
                                        mode='max',
                                        patience=hp.es_patience, 
                                        restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(
                factor=hp.lr_decay_factor,
                patience=hp.lr_patience,
                min_delta=hp.lr_min_delta,
                min_lr=hp.min_lr,
                verbose=1,
            )
    ]
    
    ).history

    last_learning_rate = history["learning_rate"][-1]
    return model, history

In [10]:
vgg19 = tfk.applications.VGG19(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(96,96,3),
    pooling='avg'
)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


## Train the classifier weights before Fine Tuning

In [ ]:
model = build_model(hp, vgg19)

vgg19.trainable = False

model, history = fit(model, augmented_train_dataset, augmented_val_dataset, hp)

Epoch 1/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 112s 1s/step - accuracy: 0.1462 - loss: 2.3163 - val_accuracy: 0.2396 - val_loss: 2.1174 - learning_rate: 3.0000e-05
Epoch 2/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 42s 580ms/step - accuracy: 0.2650 - loss: 2.1205 - val_accuracy: 0.3156 - val_loss: 1.9691 - learning_rate: 3.0000e-05
Epoch 3/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 42s 580ms/step - accuracy: 0.3451 - loss: 1.9632 - val_accuracy: 0.3530 - val_loss: 1.8189 - learning_rate: 3.0000e-05
Epoch 4/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 42s 579ms/step - accuracy: 0.3854 - loss: 1.8185 - val_accuracy: 0.3689 - val_loss: 1.7518 - learning_rate: 3.0000e-05
Epoch 5/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 42s 580ms/step - accuracy: 0.4225 - loss: 1.7080 - val_accuracy: 0.3731 - val_loss: 1.7517 - learning_rate: 3.0000e-05
Epoch 6/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 42s 579ms/step - accuracy: 0.4508 - loss: 1.6405 - val_accuracy: 0.3829 - val_loss: 1.7450 - learning_rate: 3.0000e-05
Epoch 7/20
73/73 ━━━━━━━━━━━━━━━━━━━━ 42s 580ms/step - accur

## Fine Tuning on the last 4 layers of the feature extractor

In [ ]:
vgg19.trainable = True

for layer in vgg19.layers[:15]:
    layer.trainable = False    

hp.learning_rate = last_learning_rate / 10

model, history = fit()

## Fine Tuning on the last 3 layers

In [ ]:
vgg19.trainable = True

for layer in vgg19.layers[:12]:
    layer.trainable = False    

fit()

### Save the model

In [ ]:
output_path = "/kaggle/working/vgg19-lion-wdY.keras"

model.save(output_path)